In [1]:
import duckdb

In [2]:
rel = duckdb.sql('SELECT * FROM range(1000000) tbl(id)');
rel.show()

┌────────────────────────┐
│           id           │
│         int64          │
├────────────────────────┤
│                      0 │
│                      1 │
│                      2 │
│                      3 │
│                      4 │
│                      5 │
│                      6 │
│                      7 │
│                      8 │
│                      9 │
│                      · │
│                      · │
│                      · │
│                   9990 │
│                   9991 │
│                   9992 │
│                   9993 │
│                   9994 │
│                   9995 │
│                   9996 │
│                   9997 │
│                   9998 │
│                   9999 │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘



In [3]:
duckdb.sql('SELECT COUNT(*) FROM rel').show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1000000 │
└──────────────┘



In [4]:
duckdb.sql('SELECT SUM(id) FROM rel').show()

┌──────────────┐
│   sum(id)    │
│    int128    │
├──────────────┤
│ 499999500000 │
└──────────────┘



# Operations

In [4]:
# Aggregation 
# Automatically considers standards fields as group fields
rel.aggregate('id % 3 AS g2, sum(id), min(id), max(id), count(*)')

┌───────┬──────────────┬─────────┬─────────┬──────────────┐
│  g2   │   sum(id)    │ min(id) │ max(id) │ count_star() │
│ int64 │    int128    │  int64  │  int64  │    int64     │
├───────┼──────────────┼─────────┼─────────┼──────────────┤
│     0 │ 166666833333 │       0 │  999999 │       333334 │
│     1 │ 166666166667 │       1 │  999997 │       333333 │
│     2 │ 166666500000 │       2 │  999998 │       333333 │
└───────┴──────────────┴─────────┴─────────┴──────────────┘

In [6]:
# Exclusion

r1 = duckdb.sql('SELECT * FROM range(10) tbl(id)');
r1.show()
print("Removing 6, 7, 8")
r2 = duckdb.sql('SELECT 6 UNION SELECT 7 UNION SELECT 8');
r1.except_(r2).show()


┌───────┐
│  id   │
│ int64 │
├───────┤
│     0 │
│     1 │
│     2 │
│     3 │
│     4 │
│     5 │
│     6 │
│     7 │
│     8 │
│     9 │
└───────┘

Removing 6, 7, 8
┌───────┐
│  id   │
│ int64 │
├───────┤
│     0 │
│     1 │
│     2 │
│     3 │
│     4 │
│     5 │
│     9 │
└───────┘



In [7]:
# Filter and Limit

rel.filter('id > 5').limit(3).show()

┌───────┐
│  id   │
│ int64 │
├───────┤
│     6 │
│     7 │
│     8 │
└───────┘



In [8]:
# Intersection 

r1 = duckdb.sql('SELECT * FROM range(10) tbl(r1)');
r1.show()
r2 = duckdb.sql('SELECT * FROM range(8,13) tbl(r2)');
r2.show()
print("intersecting r1 to r2")
r1.intersect(r2).show()

┌───────┐
│  r1   │
│ int64 │
├───────┤
│     0 │
│     1 │
│     2 │
│     3 │
│     4 │
│     5 │
│     6 │
│     7 │
│     8 │
│     9 │
└───────┘

┌───────┐
│  r2   │
│ int64 │
├───────┤
│     8 │
│     9 │
│    10 │
│    11 │
│    12 │
└───────┘

intersecting r1 to r2
┌───────┐
│  r1   │
│ int64 │
├───────┤
│     8 │
│     9 │
└───────┘



In [9]:
# Join
sql = """
 SELECT 1 AS id, 'vincent' AS name 
 UNION 
 SELECT 2 AS id, 'toto' AS name
"""
r1 = duckdb.sql(sql).set_alias('r1')
r1.show()

sql = """
 SELECT 1 AS id, 'busy' AS status 
 UNION 
 SELECT 2 AS id, 'free' AS status
 UNION 
 SELECT 3 AS id, 'free' AS status
"""
r2 = duckdb.sql(sql).set_alias('r2')
r2.show()

rJoined = r1.join(r2, 'r1.id = r2.id')
rJoined.show()

┌───────┬─────────┐
│  id   │  name   │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ vincent │
│     2 │ toto    │
└───────┴─────────┘

┌───────┬─────────┐
│  id   │ status  │
│ int32 │ varchar │
├───────┼─────────┤
│     3 │ free    │
│     1 │ busy    │
│     2 │ free    │
└───────┴─────────┘

┌───────┬─────────┬───────┬─────────┐
│  id   │  name   │  id   │ status  │
│ int32 │ varchar │ int32 │ varchar │
├───────┼─────────┼───────┼─────────┤
│     1 │ vincent │     1 │ busy    │
│     2 │ toto    │     2 │ free    │
└───────┴─────────┴───────┴─────────┘



In [10]:
# order 
rJoined.order('name ASC').show()

┌───────┬─────────┬───────┬─────────┐
│  id   │  name   │  id   │ status  │
│ int32 │ varchar │ int32 │ varchar │
├───────┼─────────┼───────┼─────────┤
│     2 │ toto    │     2 │ free    │
│     1 │ vincent │     1 │ busy    │
└───────┴─────────┴───────┴─────────┘



In [47]:
# project
rJoined.project('r1.id id, name, status').show()

┌───────┬─────────┬─────────┐
│  id   │  name   │ status  │
│ int32 │ varchar │ varchar │
├───────┼─────────┼─────────┤
│     1 │ vincent │ busy    │
│     2 │ toto    │ free    │
└───────┴─────────┴─────────┘



In [11]:
# project
rJoined.project('r1.id id, name, status, r1.id+10 as id2').show()

┌───────┬─────────┬─────────┬───────┐
│  id   │  name   │ status  │  id2  │
│ int32 │ varchar │ varchar │ int32 │
├───────┼─────────┼─────────┼───────┤
│     1 │ vincent │ busy    │    11 │
│     2 │ toto    │ free    │    12 │
└───────┴─────────┴─────────┴───────┘



In [12]:
# union
r1 = duckdb.sql('SELECT * FROM range(5) tbl(r1)');
r1.show()
r2 = duckdb.sql('SELECT * FROM range(20,25) tbl(r2)');
r2.show()
print("union of r1 and r2")
r1.union(r2).show()

┌───────┐
│  r1   │
│ int64 │
├───────┤
│     0 │
│     1 │
│     2 │
│     3 │
│     4 │
└───────┘

┌───────┐
│  r2   │
│ int64 │
├───────┤
│    20 │
│    21 │
│    22 │
│    23 │
│    24 │
└───────┘

union of r1 and r2
┌───────┐
│  r1   │
│ int64 │
├───────┤
│     0 │
│     1 │
│     2 │
│     3 │
│     4 │
│    20 │
│    21 │
│    22 │
│    23 │
│    24 │
└───────┘

